### Process testing file

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sb
import datetime as dt
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from math import *
import json

#### Load Dataset

In [2]:
df_test = pd.read_csv("test.csv")

In [3]:
# Get Keyset

In [4]:
df_key = df_test[['key']]

In [5]:
df_key

,key
0,2015-01-27 13:08:24.0000002
1,2015-01-27 13:08:24.0000003
2,2011-10-08 11:53:44.0000002
3,2012-12-01 21:12:12.0000002
4,2012-12-01 21:12:12.0000003
...,...
9909,2015-05-10 12:37:51.0000002
9910,2015-01-12 17:05:51.0000001
9911,2015-04-19 20:44:15.0000001
9912,2015-01-31 01:05:19.0000005


In [6]:
df_key.to_csv('test_keys.csv', index=False)

#### Get date, time, day, weekend

In [7]:
datetime_list = []
for datetime_str in df_test['pickup_datetime']:
    datetime_value = dt.datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S UTC')
    date = datetime_value.date() 
    time = datetime_value.time()
    day = datetime_value.weekday()
    weekend = (day>=4)
    datetime_list.append([date,time,day,weekend])
    #print(date)

In [8]:
date_series = pd.Series(v[0] for v in datetime_list)
time_series = pd.Series(v[1] for v in datetime_list)
day_series = pd.Series(v[2] for v in datetime_list)
weekend_series = pd.Series(v[3] for v in datetime_list)

In [9]:
cal = calendar()
holidays = cal.holidays(start= date_series.min(), end=date_series.max())
holidays = [x.date() for x in holidays]
print("Earliest Date in Train Datatset:",date_series.min())
print("Latest Date in Train Dataset:",date_series.max())

Earliest Date in Train Datatset: 2009-01-01
Latest Date in Train Dataset: 2015-06-30


In [10]:
holiday_list = []
for date in date_series: 
    if date in holidays:
        holiday_list.append(int(1))
    else: 
        holiday_list.append(int(0))   
holiday_series = pd.Series(v for v in holiday_list)

In [11]:
df_test['date'] = date_series
df_test['time'] = time_series
df_test['day'] = day_series
df_test['weekend'] = weekend_series
df_test['holiday'] = holiday_series

In [12]:
df_test.head(20)

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,date,time,day,weekend,holiday
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1,2015-01-27,13:08:24,1,False,0
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1,2015-01-27,13:08:24,1,False,0
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1,2011-10-08,11:53:44,5,True,0
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1,2012-12-01,21:12:12,5,True,0
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1,2012-12-01,21:12:12,5,True,0
5,2012-12-01 21:12:12.0000005,2012-12-01 21:12:12 UTC,-73.960983,40.765547,-73.979177,40.740053,1,2012-12-01,21:12:12,5,True,0
6,2011-10-06 12:10:20.0000001,2011-10-06 12:10:20 UTC,-73.949013,40.773204,-73.959622,40.770893,1,2011-10-06,12:10:20,3,False,0
7,2011-10-06 12:10:20.0000003,2011-10-06 12:10:20 UTC,-73.777282,40.646636,-73.985083,40.759368,1,2011-10-06,12:10:20,3,False,0
8,2011-10-06 12:10:20.0000002,2011-10-06 12:10:20 UTC,-74.014099,40.709638,-73.995106,40.741365,1,2011-10-06,12:10:20,3,False,0
9,2014-02-18 15:22:20.0000002,2014-02-18 15:22:20 UTC,-73.969582,40.765519,-73.980686,40.770725,1,2014-02-18,15:22:20,1,False,0


In [13]:
df_test.drop(columns=['key','pickup_datetime'])

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,date,time,day,weekend,holiday
0,-73.973320,40.763805,-73.981430,40.743835,1,2015-01-27,13:08:24,1,False,0
1,-73.986862,40.719383,-73.998886,40.739201,1,2015-01-27,13:08:24,1,False,0
2,-73.982524,40.751260,-73.979654,40.746139,1,2011-10-08,11:53:44,5,True,0
3,-73.981160,40.767807,-73.990448,40.751635,1,2012-12-01,21:12:12,5,True,0
4,-73.966046,40.789775,-73.988565,40.744427,1,2012-12-01,21:12:12,5,True,0
...,...,...,...,...,...,...,...,...,...,...
9909,-73.968124,40.796997,-73.955643,40.780388,6,2015-05-10,12:37:51,6,True,0
9910,-73.945511,40.803600,-73.960213,40.776371,6,2015-01-12,17:05:51,0,False,0
9911,-73.991600,40.726608,-73.789742,40.647011,6,2015-04-19,20:44:15,6,True,0
9912,-73.985573,40.735432,-73.939178,40.801731,6,2015-01-31,01:05:19,5,True,0


#### Check if peak hour

In [14]:
peak_hour_df= df_test[['time','day']]
peak_hour_df

,time,day
0,13:08:24,1
1,13:08:24,1
2,11:53:44,5
3,21:12:12,5
4,21:12:12,5
...,...,...
9909,12:37:51,6
9910,17:05:51,0
9911,20:44:15,6
9912,01:05:19,5


In [15]:
# Initalise peak hour timings
def in_between(now, start, end):
    if start <= end:
        return start < now <= end
    else: # over midnight e.g., 23:30-04:15
        return start < now or now <= end
    
    
morning_from = dt.time(7,0,0)
morning_till = dt.time(9,0,0)

evening_from = dt.time(15,0,0)
evening_till = dt.time(19,0,0)

evening_from_fri = dt.time(14,0,0)

peak_hour_list = []
for index, row in peak_hour_df.iterrows():
    #time = row['time']
    time = row['time']
    
    # If day is Mon - Thur ------------
    if 0 <= row['day'] <= 3:
        if in_between(time, morning_from, morning_till) or in_between(time, evening_from, evening_till):
            peak_hour_list.append(int(1))
        else:
            peak_hour_list.append(int(0))
    # If day is Friday -----------------
    
    elif row['day'] == 4: 
        if in_between(time, morning_from, morning_till) or in_between(time, evening_from_fri, evening_till):
            peak_hour_list.append(int(1))
        else:
            peak_hour_list.append(int(0))
    # If day is Sat - Sun 
    else: 
        peak_hour_list.append(int(0)) 

In [16]:
peak_hour_series = pd.Series(v for v in peak_hour_list)

In [17]:
peak_hour_df

,time,day
0,13:08:24,1
1,13:08:24,1
2,11:53:44,5
3,21:12:12,5
4,21:12:12,5
...,...,...
9909,12:37:51,6
9910,17:05:51,0
9911,20:44:15,6
9912,01:05:19,5


In [18]:
df_test['peak_hour'] = peak_hour_series

#### Get Hotspot & get Dist

In [19]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [20]:
hot_dist_df = df_test[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']]
hotspot_json = pd.read_json('hotspot_location.json')
hotspot_json

,times_square,grand_central,memorial,rockefeller,union_square,central_park,soho,highline,tompkins,brooklyn_museum
area1,"[40.76246, -73.989779]","[40.756996, -73.978261]","[40.715161, -74.015973]","[40.761666, -73.982854]","[40.73978, -73.995001]","[40.792898, -73.967833]","[40.722215, -74.00072]","[40.749751, -74.007163]","[40.728263, -73.983154]","[40.675063, -73.96672]"
area2,"[40.753007, -73.979601]","[40.748831, -73.973815]","[40.708709999999996, -74.010904]","[40.757262, -73.978071]","[40.731591, -73.986553]","[40.771863, -73.963416]","[40.718183, -73.997963]","[40.744284, -74.001868]","[40.724239, -73.981092]","[40.670381, -73.962357]"


In [21]:
hot_dist_df

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,-73.973320,40.763805,-73.981430,40.743835
1,-73.986862,40.719383,-73.998886,40.739201
2,-73.982524,40.751260,-73.979654,40.746139
3,-73.981160,40.767807,-73.990448,40.751635
4,-73.966046,40.789775,-73.988565,40.744427
...,...,...,...,...
9909,-73.968124,40.796997,-73.955643,40.780388
9910,-73.945511,40.803600,-73.960213,40.776371
9911,-73.991600,40.726608,-73.789742,40.647011
9912,-73.985573,40.735432,-73.939178,40.801731


In [22]:
# Latitude is the 40.721319 (Positive number) while longitude is the negative number
def within_area(given_lon, given_lat,one_lon,one_lat,two_lon, two_lat):
    if one_lon <= given_lon <= two_lon and one_lat >= given_lat >= two_lat: 
        return True
    else: 
        return False

In [23]:
hotspot_list = []
for index, row in hot_dist_df.iterrows():
    for x in hotspot_json.keys(): 
        if within_area(row['pickup_longitude'], row['pickup_latitude'], hotspot_json[x]['area1'][1], hotspot_json[x]['area1'][0], hotspot_json[x]['area2'][1], hotspot_json[x]['area2'][0]) or within_area(row['dropoff_longitude'], row['dropoff_latitude'], hotspot_json[x]['area1'][1], hotspot_json[x]['area1'][0], hotspot_json[x]['area2'][1], hotspot_json[x]['area2'][0]):
            hotspot = int(1)
            break
        else: 
            hotspot = int(0)
            
    hotspot_list.append(hotspot)


In [24]:
hotspot_series = pd.Series(v for v in hotspot_list)
hotspot_series

0       0
1       0
2       0
3       0
4       1
       ..
9909    0
9910    0
9911    0
9912    0
9913    1
Length: 9914, dtype: int64

In [25]:
hotspot_series.value_counts()

0    7604
1    2310
dtype: int64

In [26]:
df_test['hotspot'] = hotspot_series

In [27]:
dist_list = []
for index, row in hot_dist_df.iterrows():
    dist = haversine(row['pickup_longitude'], row['pickup_latitude'],row['dropoff_longitude'],row['dropoff_latitude'])
    dist_list.append(dist)

In [28]:
dist_series = pd.Series(v for v in dist_list)
dist_series

0        2.323260
1        2.425353
2        0.618628
3        1.961033
4        5.387301
          ...    
9909     2.124874
9910     3.270969
9911    19.183941
9912     8.343486
9913     1.180825
Length: 9914, dtype: float64

In [29]:
df_test['dist'] = dist_series

In [30]:
df_test

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,date,time,day,weekend,holiday,peak_hour,hotspot,dist
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1,2015-01-27,13:08:24,1,False,0,0,0,2.323260
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1,2015-01-27,13:08:24,1,False,0,0,0,2.425353
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1,2011-10-08,11:53:44,5,True,0,0,0,0.618628
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1,2012-12-01,21:12:12,5,True,0,0,0,1.961033
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1,2012-12-01,21:12:12,5,True,0,0,1,5.387301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,2015-05-10 12:37:51 UTC,-73.968124,40.796997,-73.955643,40.780388,6,2015-05-10,12:37:51,6,True,0,0,0,2.124874
9910,2015-01-12 17:05:51.0000001,2015-01-12 17:05:51 UTC,-73.945511,40.803600,-73.960213,40.776371,6,2015-01-12,17:05:51,0,False,0,1,0,3.270969
9911,2015-04-19 20:44:15.0000001,2015-04-19 20:44:15 UTC,-73.991600,40.726608,-73.789742,40.647011,6,2015-04-19,20:44:15,6,True,0,0,0,19.183941
9912,2015-01-31 01:05:19.0000005,2015-01-31 01:05:19 UTC,-73.985573,40.735432,-73.939178,40.801731,6,2015-01-31,01:05:19,5,True,0,0,0,8.343486


In [31]:
processed_test = df_test.drop(columns=['pickup_datetime','date'])

In [32]:
processed_test.to_csv('processed_test.csv', index=False)

In [33]:
processed_test

,key,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,time,day,weekend,holiday,peak_hour,hotspot,dist
0,2015-01-27 13:08:24.0000002,-73.973320,40.763805,-73.981430,40.743835,1,13:08:24,1,False,0,0,0,2.323260
1,2015-01-27 13:08:24.0000003,-73.986862,40.719383,-73.998886,40.739201,1,13:08:24,1,False,0,0,0,2.425353
2,2011-10-08 11:53:44.0000002,-73.982524,40.751260,-73.979654,40.746139,1,11:53:44,5,True,0,0,0,0.618628
3,2012-12-01 21:12:12.0000002,-73.981160,40.767807,-73.990448,40.751635,1,21:12:12,5,True,0,0,0,1.961033
4,2012-12-01 21:12:12.0000003,-73.966046,40.789775,-73.988565,40.744427,1,21:12:12,5,True,0,0,1,5.387301
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,-73.968124,40.796997,-73.955643,40.780388,6,12:37:51,6,True,0,0,0,2.124874
9910,2015-01-12 17:05:51.0000001,-73.945511,40.803600,-73.960213,40.776371,6,17:05:51,0,False,0,1,0,3.270969
9911,2015-04-19 20:44:15.0000001,-73.991600,40.726608,-73.789742,40.647011,6,20:44:15,6,True,0,0,0,19.183941
9912,2015-01-31 01:05:19.0000005,-73.985573,40.735432,-73.939178,40.801731,6,01:05:19,5,True,0,0,0,8.343486
